In [1]:
import os
import numpy as np
import codecs
path_baptiste = "/home/baptiste/Documents/data/train"
path_igor = "C:/Users/Igor/Documents/Master Data Science/Big Data Analytics/Projet/Data/train"
path_sofia = "/Users/Flukmacdesof/data 2/train"
path_sofia_server = "/home/sofia.calcagno/data 2/train"


#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with codecs.open(rootdirPOS+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with codecs.open(rootdirNEG+"/"+file, 'r',encoding="utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with open(rootdir+"/"+file, 'r', endoding= "utf-8") as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

## Data loading

In [2]:
reviews, Class = loadLabeled(path_sofia_server)

In [3]:
reviews_sample = np.concatenate([reviews[:100], reviews[-100:]])


In [48]:
Class_sample = np.concatenate([Class[:100], Class[-100:]])

In [32]:
from nltk.stem.snowball import SnowballStemmer
import string
from nltk.corpus import stopwords
stemmer = SnowballStemmer("english")
def data_preparation_twidf(reviews):
    HTMLlist = ['<br />'] # list of HLML signs
    exclude = set(string.punctuation) #list of punctuation 
    swords = stopwords.words("english") #list of stopwords
    for idx, review in enumerate(reviews):
        for word in HTMLlist:
            reviews[idx] = review.replace(word,' ') # Remove HLML signs
        reviews[idx] = ''.join([stemmer.stem(word)+' ' for word in reviews[idx].split()]) #stemming
        reviews[idx] = ''.join([w if w not in exclude else " " for w in reviews[idx].lower() ])
        reviews[idx] = ''.join([w +' ' for w in reviews[idx].split() if w not in swords])
    return reviews


In [33]:
clean_reviews = data_preparation_twidf(reviews)
clean_sample_reviews = data_preparation_twidf(reviews_sample)

In [34]:
unique_words=list(set(''.join([review for review in clean_reviews ]).split(' ')))
unique_words_sample=list(set(''.join([review for review in clean_sample_reviews ]).split(' ')))
num_documents = len(reviews)
num_documents_sample = len(reviews_sample)
sliding_window = 2
train_par = True
idf_learned = {}

In [35]:
import networkx as nx
def createGraphFeatures(num_documents,clean_train_documents,unique_words,sliding_window,train_par,idf_learned):
	features = np.zeros((num_documents,len(unique_words)))#where we are going to put the features
	unique_words_len = len(unique_words)
	term_num_docs = {} #dictionay of each word with a count of that word through out the collections
	idf_col = {}#dictionay of each word with the idf of that word
	
	#TO DO:
	#1.idf_col:IDF for the collection
	#	if in training phase compute it
	#	else use the one provided
	#2. term_num_docs : count of the words in the collection
	#	if in training phase populate it
	#	else use the one provided
	if train_par:
		
		#for all documents
		for i in range( 0,num_documents ):
			if i%100 == 0 :
				print "idf", i
			wordList = clean_train_documents[i].split(None)
			#count word occurrences through the collection (for idf) put the count in term_num_docs
			if len(wordList)>1:
				countWords(wordList,term_num_docs) #TODO: implement this function 
		#TODO: calculate the idf for all words
		for term_x in term_num_docs:
			idf_col[term_x] = np.log10(float(num_documents)/term_num_docs[term_x])            
	# for the testing set
	else:
		#use the existing ones if we are in the test data
		idf_col = idf_learned 
		term_num_docs=unique_words

	print "Creating the graph of words for each document..."
	totalNodes = 0
	totalEdges = 0

	#go over all documents
	for i in range( 0,num_documents ):
		if i%100 == 0 :
			print "tw", i
		wordList = clean_train_documents[i].split(None)
		docLen = len(wordList)
		#the graph
		dG = nx.Graph()

		if len(wordList)>1:
			populateGraph(wordList,dG,sliding_window)
			dG.remove_edges_from(dG.selfloop_edges())
			centrality = nx.degree_centrality(dG) #dictionary of centralities (node:degree)

			totalNodes += dG.number_of_nodes()
			totalEdges += dG.number_of_edges()
			#TODO : implement comments bellow
			# for all nodes
				#If they are in the desired features
					#compute the TW-IDF score and put it in features[i,unique_words.index(g)]
			for k, node_term in enumerate(dG.nodes()):
				if node_term in idf_col:
					features[i,unique_words.index(node_term)] = centrality[node_term] * idf_col[node_term]

	if train_par:
		nodes_ret=term_num_docs.keys()
		#print "Percentage of features kept:"+str(feature_reduction)
		print "Average number of nodes:"+str(float(totalNodes)/num_documents)
		print "Average number of edges:"+str(float(totalEdges)/num_documents)
	else:
		nodes_ret=term_num_docs
	#return 1: features, 2: idf values (for the test data), 3: the list of terms 
	return features, idf_col, nodes_ret

In [36]:
def populateGraph(wordList,dG,sliding_window):
	#TODO: implement this function
	#For each position/word in the word list:
		#add the -new- word in the graph
		#for all words -forward- within the window size
			#add new words as new nodes 
			#add edges among all word within the window
	for k, word in enumerate(wordList):
		if not dG.has_node(word):
			dG.add_node(word)
		if k+sliding_window > len(wordList):
			tempW=len(wordList) - k
		else:
			tempW=sliding_window
		for j in xrange(1, tempW):
			next_word = wordList[k+j]
			dG.add_edge(word, next_word)

In [37]:
def countWords(wordList,term_num_docs):
	found = set()
	#TODO: implement this function
	#add the terms from the wordlist to the term_num_docs dictionary or increase its count
	for k, word in enumerate(wordList):
		if word not in found:
			found.add(word)
			if word in term_num_docs:
				term_num_docs[word] +=1
			else :
				term_num_docs[word] = 1

In [38]:
tw_idf=createGraphFeatures(num_documents_sample,clean_sample_reviews,unique_words_sample,sliding_window,train_par,idf_learned)

idf 0
idf 100
Creating the graph of words for each document...
tw 0
tw 100
Average number of nodes:96.905
Average number of edges:116.435


In [42]:
import scipy

In [45]:
sparse_tw_idf = scipy.sparse.csr_matrix(tw_idf[0])

##  CSV Creation:

Create 5 csv : train_train.csv, train_test.csv, y_train_train.csv, y_train_test.csv, test.csv

In [49]:
# Split the tf-idf matrix into two data sets to process the cross validation : training and test set
from sklearn.cross_validation import train_test_split

data_train, data_test, label_train, label_test = train_test_split(sparse_tw_idf, Class_sample, test_size = 0.3, random_state = 42)

##### 

In [50]:
import csv

def save_sparse_csr(filename, array):
    np.savez(filename, data = array.data, indices = array.indices, 
             indptr = array.indptr, shape = array.shape)
    
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix(( loader['data'], loader['indices'], loader['indptr']),
                     shape = loader['shape'])

def save_csv(filename, array):
    with open(filename, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter = '\n')
        writer.writerow(array)
        
def load_csv(filename):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter = '\n')
        array = [float(row[0]) for row in reader]
        return array

In [52]:
save_sparse_csr('data_tw_train', data_train)
save_sparse_csr('data_tw_test', data_test)
save_csv('label_tw_train.csv', label_train)
save_csv('label_tw_test.csv', label_test)